<a href="https://colab.research.google.com/github/data2450/wine-qulity-prediction-ML/blob/main/full_wine_qulity_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
url='https://raw.githubusercontent.com/data2450/wine-qulity-prediction-ML/main/winequality-red.csv'

In [ ]:
import numpy as np # linear algebra
import pandas as pd# data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
wine=pd.read_csv(url)
print(wine.info())
wine.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB
None


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [ ]:
wine1=wine[['chlorides','alcohol','quality']]
wine1.head()

,chlorides,alcohol,quality
0,0.076,9.4,5
1,0.098,9.8,5
2,0.092,9.8,5
3,0.075,9.8,6
4,0.076,9.4,5


In [ ]:
#preprocessing
#Making binary classificaion for the response variable.
#Dividing wine as good and bad by giving the limit for the quality
bins = (2, 6.5, 8)
group_names = ['bad', 'good']
wine1['quality'] = pd.cut(wine1['quality'], bins = bins, labels = group_names)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
#Now lets assign a labels to our quality variable
label_quality = LabelEncoder()

In [ ]:
#Bad becomes 0 and good becomes 1 
wine1['quality'] = label_quality.fit_transform(wine1['quality'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
data=np.array(wine1)

In [ ]:
#Now seperate the dataset as response variable and feature variabes
X =data[1:,0:-1]
y=data[1:,-1]

In [ ]:
X=X.astype('int')
y=y.astype('int')

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
#Train and Test splitting of data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
#Applying Standard scaling to get optimized result
sc = StandardScaler()

In [ ]:
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [ ]:
#model selection
#applying random forest
from sklearn.model_selection import KFold
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
from sklearn.ensemble import RandomForestClassifier
rnd = RandomForestClassifier(n_estimators=80)
scoring = 'accuracy'
score = cross_val_score(rnd, X_train, y_train, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

[0.8359375  0.796875   0.90625    0.8515625  0.90625    0.8984375
 0.8671875  0.859375   0.88188976 0.80314961]


In [ ]:
rnd=rnd.fit(X_train,y_train)

In [ ]:
import pickle
# open a file, where you ant to store the data
file = open('model.pkl', 'wb')
# dump information to that file
pickle.dump(rnd, file)

In [ ]:
#from sklearn.metrics import confusion_matrix, classification_report
#print(classification_report(y_test, prediction))
wine1.columns


Index(['chlorides', 'alcohol', 'quality'], dtype='object')

In [ ]:
!pip install streamlit

In [ ]:
%%writefile app.py
import streamlit as st
import pickle
import numpy as np
model=pickle.load(open('model.pkl','rb'))


def predict_forest(chlorides,alcohol):
    input=np.array([[chlorides,alcohol]]).astype(np.float64)
    prediction=model.predict_proba(input)
    pred='{0:.{1}f}'.format(prediction[0][0],2)
    return float(pred)

def main():
    st.title("Streamlit Tutorial")
    html_temp = """
    <div style="background-color:#025246 ;padding:10px">
    <h2 style="color:white;text-align:center;">wine quality Prediction ML App </h2>
    </div>
    """
    st.markdown(html_temp, unsafe_allow_html=True)

    
    chlorides = st.text_input("chlorides","Type Here")
    alcohol = st.text_input("alcohol","Type Here")
    safe_html="""  
      <div style="background-color:#F4D03F;padding:10px >
       <h2 style="color:white;text-align:center;"> Your wine is good</h2>
       </div>
    """
    danger_html="""  
      <div style="background-color:#F08080;padding:10px >
       <h2 style="color:black ;text-align:center;"> Your wine is in bad</h2>
       </div>
    """

    if st.button("Predict"):
        output=predict_forest(chlorides,alcohol)
        st.success('The probability of wine being bad is {}'.format(output))

        if output > 0.5:
            st.markdown(safe_html,unsafe_allow_html=True)
        else:
            st.markdown(danger_html,unsafe_allow_html=True)

if __name__=='__main__':
    main()

Overwriting app.py


In [ ]:
!pip install pyngrok

In [ ]:
!ngork authtoken 1j2f85MYT4zBqW0XgGZ9b21Pa8U_4WSSSn9da9HYd4cePzwZc

/bin/bash: ngork: command not found


In [ ]:
!nohup streamlit run app.py &

nohup: appending output to 'nohup.out'


In [ ]:
from pyngrok import ngrok
url=ngrok.connect(port=8501)
url